In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import shutil
import random
import torch

In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())  # Should be True
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("CUDA Version:", torch.version.cuda)  # Should print a version number
print(torch.cuda.device_count())
print("PyTorch Version:", torch.__version__)  # Should show `+cu118` or `+cu121`

CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 11.8
1
PyTorch Version: 2.6.0+cu118


In [4]:
# Define the dataset directory
dataset_dir = r"C:\Users\Bisoj Pc\Desktop\test1_wasteNot"
image_dir = os.path.join(dataset_dir, "images")  # Images folder
label_dir = os.path.join(dataset_dir, "labels")  # Labels folder

In [5]:
# Define output directories for train, val, test
output_dirs = {
    "train": {"images": os.path.join(dataset_dir, "images/train"), "labels": os.path.join(dataset_dir, "labels/train")},
    "val": {"images": os.path.join(dataset_dir, "images/val"), "labels": os.path.join(dataset_dir, "labels/val")},
    "test": {"images": os.path.join(dataset_dir, "images/test"), "labels": os.path.join(dataset_dir, "labels/test")},
}

In [6]:
# Create output directories if they don't exist
for split in output_dirs.values():
    os.makedirs(split["images"], exist_ok=True)
    os.makedirs(split["labels"], exist_ok=True)

In [7]:
# Get all images and ensure corresponding labels exist
image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
paired_files = [(img, img.replace(".jpg", ".txt")) for img in image_files if os.path.exists(os.path.join(label_dir, img.replace(".jpg", ".txt")))]

In [8]:
# Shuffle and split dataset
random.shuffle(paired_files)
train_split = int(0.8 * len(paired_files))
val_split = int(0.9 * len(paired_files))

train_files = paired_files[:train_split]
val_files = paired_files[train_split:val_split]
test_files = paired_files[val_split:]

In [9]:
# Function to move files
def move_files(file_pairs, split):
    for img_file, label_file in file_pairs:
        shutil.move(os.path.join(image_dir, img_file), os.path.join(output_dirs[split]["images"], img_file))
        shutil.move(os.path.join(label_dir, label_file), os.path.join(output_dirs[split]["labels"], label_file))

In [10]:
# Move files to respective directories
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")


print(f"Dataset split complete! ✅")
print(f"Train: {len(train_files)} images")
print(f"Validation: {len(val_files)} images")
print(f"Test: {len(test_files)} images")

Dataset split complete! ✅
Train: 0 images
Validation: 0 images
Test: 0 images


In [11]:
import os
import cv2

dataset_path = "D:\\WasteNot\\"
image_exts = {".jpg", ".jpeg", ".png"}

for split in ["train", "val", "test"]:
    image_dir = os.path.join(dataset_path, "images", split)
    label_dir = os.path.join(dataset_path, "labels", split)

    images = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if os.path.splitext(f)[1].lower() in image_exts}
    labels = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith(".txt")}

    # Check for missing images or labels
    missing_images = labels - images
    missing_labels = images - labels

    if missing_images:
        print(f"⚠️ Missing Images in {split}: {missing_images}")
    if missing_labels:
        print(f"⚠️ Missing Labels in {split}: {missing_labels}")

    # Check for corrupted images
    for img_name in images:
        img_path = os.path.join(image_dir, img_name + ".jpg")  # Change extension if needed
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Corrupted Image: {img_path}")

print("✅ Dataset check complete!")


KeyboardInterrupt: 

In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from ultralytics import YOLO
import torch

# Ensure that CUDA is available and set the device to GPU 1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use GPU 1 (second GPU)

# Load the YOLOv8l model
model = YOLO("yolov8l.pt")  # Using pre-trained weights

# Explicitly force the model to use GPU 1 (cuda:1)
model.to('cuda:0')  # Moves the model to GPU 1

print("Using device:", model.device)



Using device: cuda:0


In [11]:
results = model.train(
    data="dataset.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=0.005,
    lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3 ,
    device="cuda:0",
    cos_lr=True,  # Added
)

New https://pypi.org/project/ultralytics/8.3.80 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.79  Python-3.13.1 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8l.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning C:\Users\Bisoj Pc\Desktop\test1_wasteNot\labels\train.cache... 159916 images, 0 backgrounds, 0 corrupt: 100%|██████████| 159916/159916 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\Bisoj Pc\Desktop\test1_wasteNot\labels\val.cache... 19989 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19989/19989 [00:00<?, ?it/s]


Plotting labels to runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      9.21G     0.9225      2.624      1.391         29        640:   3%|▎         | 325/9995 [44:25<24:54:55,  9.28s/it]